In [17]:
import pandas as pd
import glob
import os

# Specify the folder path where CSV files are stored
folder_path = 'https://github.com/SunMax44/football_match_predictor/tree/main/data'

# Use glob to find all CSV files in the specified folder
all_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Use a list comprehension to read each CSV file into a DataFrame
#df_list = [pd.read_csv(file) for file in all_files]

# Concatenate all DataFrames in the list into a single DataFrame
#buli_df = pd.concat(df_list, ignore_index=True)

# Display the combined DataFrame
#print(buli_df)


[]